# Tweet Collection and Storing Them

* Insert tweets into db with some id, hashing its text and user


### FoxTV Ana Haber Bülteni Hashtag'leri

* 2 Nisan 2020 : #şüpheduymuyorum
* 3 Nisan 2020 : #duysesimi
* 4 Nisan 2020 : #
* 5 Nisan 2020 : #
* 6 Nisan 2020 : #neyinnesi (Erdoğan Açıklama 19:00)
* 7 Nisan 2020 : #biralkışda
* 8 Nisan 2020 : #hepsinitopla
* 9 Nisan 2020 : #kabuledilemez
* 10 Nisan 2020 : #neredenbulacağım

In [1]:
import tweepy
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
import json

In [3]:
client = MongoClient('mongodb://localhost:27017/')

db = client.covid_hack
tweets_db = db.tweets

tweets_db.find_one()

In [5]:
file = open('keys.json')
key_dict = json.loads(file.read())
file.close()

In [6]:
auth = tweepy.OAuthHandler(key_dict['consumer_key'], key_dict['consumer_secret'])
auth.set_access_token(key_dict['access_token'], key_dict['access_token_secret'])

api = tweepy.API(auth)

In [7]:
def reshape_tweet(tweet, hashtag_id):
    tweet['hashtag_id'] = hashtag_id
    return tweet

In [8]:
searched_hashtags = ['#şüpheduymuyorum', '#duysesimi', '#biralkısda', '#neyinnesi', "#biralkışda", '#hepsinitopla']

In [9]:
tweet_search_continue = True
check_all_for_dup = True
hashtag_id = '#gerçekfikrine'
max_id = None
request_number = 0

In [ ]:
f = open('conflicts.txt', 'a+')

while tweet_search_continue:
    
    print("Request {}".format(request_number))
    
    request_number += 1
    
    tweets = api.search(q=hashtag_id, count=100, lang="tr", max_id=max_id, tweet_mode="extended")
    
    if len(tweets) == 0:
        print("No tweets fetched")
        break
    
    tweets = [tweet._json for tweet in tweets]

    print("Number of tweets {}".format(len(tweets)))
    
    tweets_filtered = []
    for tweet in tweets:
        
        # Check if it is a RT, if RT do no include
        if tweet['full_text'].startswith('RT'):
            continue
        
        no_conflict = True
        
        for hashtag in searched_hashtags:
            if hashtag in tweet['full_text']:
                print("Hashtag {} found in tweet {}".format(hashtag, tweet['id']))
                found = tweets_db.find_one({'id': tweet['id']})
                if found:
                    print("{}:{}:{}".format(hashtag_id, hashtag,tweet['id']), file=f)
                    no_conflict = False
        if no_conflict:
            tweets_filtered.append(tweet)
    
    if len(tweets_filtered) == 0:
        # Means all the fetched tweets are unfiltered as of conflict
        print("No tweets to write database")
        
        last_tweet = tweets[len(tweets)-1]
        max_id = last_tweet['id'] - 1
    
    else:
        print("{} tweets written to db".format(len(tweets_filtered)))
        try:
            tweets_db.insert_many([reshape_tweet(tweet, hashtag_id) for tweet in tweets_filtered])
        except BulkWriteError as bwe:
            pprint(bwe.details)
            raise

        last_tweet = tweets_filtered[len(tweets_filtered)-1]
        max_id = last_tweet['id'] - 1

f.close()

https://twitter.com/sercanoksak/status/1246154630175875076